# 基于车速和密度的流量预测模型

In [1]:
    import pandas as pd
    import numpy as np

In [2]:
    pd_vol_train = pd.read_csv('../../Processed_data/by_weikai/processed_train_trajectories.csv')
    pd_vol_train.head()

,intersection_id,tollgate_id,vehicle_id,starting_time,travel_seq,travel_time,date,hour,time_window,weekday,holiday,weather_hour,links
0,B,3,1065642,2016-07-19 00:14:24,105#2016-07-19 00:14:24#9.56;100#2016-07-19 00...,70.85,2016-07-19,0,"[00:00:00,00:20:00)",2,False,0,105;100;111;103;122
1,B,3,1047198,2016-07-19 00:35:56,105#2016-07-19 00:35:56#11.58;100#2016-07-19 0...,148.79,2016-07-19,0,"[00:20:00,00:40:00)",2,False,0,105;100;111;103;122
2,B,1,1086390,2016-07-19 00:37:15,105#2016-07-19 00:37:15#5.26;100#2016-07-19 00...,79.76,2016-07-19,0,"[00:20:00,00:40:00)",2,False,0,105;100;111;103;116;101;121;106;113
3,A,2,1071181,2016-07-19 00:37:59,110#2016-07-19 00:37:59#13.74;123#2016-07-19 0...,58.05,2016-07-19,0,"[00:20:00,00:40:00)",2,False,0,110;123;107;108;120;117
4,B,1,1065807,2016-07-19 00:56:21,105#2016-07-19 00:56:21#16.08;100#2016-07-19 0...,137.98,2016-07-19,0,"[00:40:00,01:00:00)",2,False,0,105;100;111;103;116;101;121;106;113


In [3]:
    pd_links_train = pd.read_csv('../../Processed_data/by_weikai/porcessed_train_link.csv')
    pd_links_train.head()

,link_id,length,width,lanes,in_top,out_top,lane_width,fork
0,100,58,3,1,105,111,3,False
1,101,84,3,1,116,121,3,False
2,102,131,9,3,115,109,3,False
3,103,23,12,4,111,"122,116",3,True
4,104,293,9,3,109,112,3,False


In [16]:
    def get_length(sequence,s):
        links = sequence.split(';')
        length = 0
        for link in links:
            l = s.loc[int(link)][0]
            length += l
        return length
    
    link_s = pd_links_train[['link_id','length']].set_index(['link_id'])
    pd_vol_train['route_length'] = pd_vol_train['links'].apply(get_length, s = link_s)
    pd_vol_train.head()
            

,intersection_id,tollgate_id,vehicle_id,starting_time,travel_seq,travel_time,date,hour,time_window,weekday,holiday,weather_hour,links,route_length
0,B,3,1065642,2016-07-19 00:14:24,105#2016-07-19 00:14:24#9.56;100#2016-07-19 00...,70.85,2016-07-19,0,"[00:00:00,00:20:00)",2,False,0,105;100;111;103;122,477
1,B,3,1047198,2016-07-19 00:35:56,105#2016-07-19 00:35:56#11.58;100#2016-07-19 0...,148.79,2016-07-19,0,"[00:20:00,00:40:00)",2,False,0,105;100;111;103;122,477
2,B,1,1086390,2016-07-19 00:37:15,105#2016-07-19 00:37:15#5.26;100#2016-07-19 00...,79.76,2016-07-19,0,"[00:20:00,00:40:00)",2,False,0,105;100;111;103;116;101;121;106;113,821
3,A,2,1071181,2016-07-19 00:37:59,110#2016-07-19 00:37:59#13.74;123#2016-07-19 0...,58.05,2016-07-19,0,"[00:20:00,00:40:00)",2,False,0,110;123;107;108;120;117,384
4,B,1,1065807,2016-07-19 00:56:21,105#2016-07-19 00:56:21#16.08;100#2016-07-19 0...,137.98,2016-07-19,0,"[00:40:00,01:00:00)",2,False,0,105;100;111;103;116;101;121;106;113,821


In [17]:
    pd_vol_train['v/min'] = pd_vol_train['route_length']*60/pd_vol_train['travel_time']
    pd_vol_train.head()

,intersection_id,tollgate_id,vehicle_id,starting_time,travel_seq,travel_time,date,hour,time_window,weekday,holiday,weather_hour,links,route_length,v/min
0,B,3,1065642,2016-07-19 00:14:24,105#2016-07-19 00:14:24#9.56;100#2016-07-19 00...,70.85,2016-07-19,0,"[00:00:00,00:20:00)",2,False,0,105;100;111;103;122,477,403.952011
1,B,3,1047198,2016-07-19 00:35:56,105#2016-07-19 00:35:56#11.58;100#2016-07-19 0...,148.79,2016-07-19,0,"[00:20:00,00:40:00)",2,False,0,105;100;111;103;122,477,192.351637
2,B,1,1086390,2016-07-19 00:37:15,105#2016-07-19 00:37:15#5.26;100#2016-07-19 00...,79.76,2016-07-19,0,"[00:20:00,00:40:00)",2,False,0,105;100;111;103;116;101;121;106;113,821,617.602808
3,A,2,1071181,2016-07-19 00:37:59,110#2016-07-19 00:37:59#13.74;123#2016-07-19 0...,58.05,2016-07-19,0,"[00:20:00,00:40:00)",2,False,0,110;123;107;108;120;117,384,396.899225
4,B,1,1065807,2016-07-19 00:56:21,105#2016-07-19 00:56:21#16.08;100#2016-07-19 0...,137.98,2016-07-19,0,"[00:40:00,01:00:00)",2,False,0,105;100;111;103;116;101;121;106;113,821,357.008262


In [25]:
    v = pd_vol_train[pd_vol_train['holiday'] == False].groupby(['intersection_id','time_window','tollgate_id'])['travel_time'].mean()
    v

intersection_id  time_window          tollgate_id
A                [00:00:00,00:20:00)  2               56.829084
                                      3              109.486389
                 [00:20:00,00:40:00)  2               54.213178
                                      3              112.902676
                 [00:40:00,01:00:00)  2               61.952500
                                      3              102.792909
                 [01:00:00,01:20:00)  2               65.605357
                                      3               95.816087
                 [01:20:00,01:40:00)  2               58.350891
                                      3               92.390426
                 [01:40:00,02:00:00)  2               52.264468
                                      3              101.842222
                 [02:00:00,02:20:00)  2               64.670750
                                      3              100.836970
                 [02:20:00,02:40:00)  2               